In [ ]:
pip install overpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 KB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for overpy: filename=overpy-0.6-py3-none-any.whl size=14106 sha256=e2e432fa78ba617d95e5f5dfc41a7ef6aaff5e59d4a278bbb14c7180008a4f11
  Stored in directory: /root/.cache/pip/wheels/69/6f/9b/c4e56235399fbce16dfcd71a65bd2af0e5882684cc4fa2ec70
Successfully built overpy


In [ ]:
import pandas as pd
pd.set_option('max_columns', 200)
from google.colab import drive 
drive.mount('/content/drive')

import time


Mounted at /content/drive


In [ ]:
import overpy
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import math

api = overpy.Overpass()
query = 'node["highway"="bus_stop"](around:500, {0}, {1}); out;'

geolocator = Nominatim(user_agent="my-application")  

address

#--------------------------------------------------------
# Command of confirming Specification 
location = geolocator.geocode("200 N Spring St. Los Angeles")

lat = location.latitude
lon = location.longitude


result = api.query(query.format(lat, lon))


closest_stop = None
closest_distance = math.inf
for node in result.nodes:
    distance = math.sqrt((float(node.lat) - lat)**2 + (float(node.lon) - lon)**2)
    if distance < closest_distance:
        closest_distance = distance
        closest_stop = node.tags.get('name', 'unknown')

print(f"The nearest bus stop is {closest_stop}, which is {closest_distance:.2f} meters away.")

(34.0537767, -118.242898)
The nearest bus stop is Higuera & Helms, which is 0.00 meters away.


In [ ]:
def NearbyFacilities(lat, lon, query):
  result = api.query(query.format(lat, lon))
  closest_distance = math.inf
  for node in result.nodes:
      distance = math.sqrt((float(node.lat) - lat)**2 + (float(node.lon) - lon)**2)
      if distance < closest_distance:
          closest_distance = distance
  return (closest_distance,len(result.nodes))


In [ ]:
data = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Capstone/data_zip.pkl')

In [ ]:
data_geo = data[["id","latitude","longitude"]]

In [ ]:
query = 'node["highway"="bus_stop"](around:500, {0}, {1}); out;'

for i in range(int(len(data)/100)):
  data_temp = data_geo.iloc[i*100:(i+1)*100,:]
  try:
    data_temp["BusStop_distance"] = data_temp.apply(lambda x: NearbyFacilities(x["latitude"],x["longitude"], query), axis = 1)
    path = "/content/drive/MyDrive/Colab Notebooks/Capstone/BusStop/data_BusStop" + str(i) + ".pkl"
    data_temp.to_pickle(path)
    time.sleep(5)
  except  :
    print("except "+ str(i))




<ipython-input-8-f8ab9fa30a41>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_temp["BusStop_distance"] = data_temp.apply(lambda x: NearbyFacilities(x["latitude"],x["longitude"], query), axis = 1)


In [ ]:
df_BusStop = pd.DataFrame(columns = ["id","latitude","longitude", "BusStop_distance"])
for i in range(405):
  path = "/content/drive/MyDrive/Colab Notebooks/Capstone/BusStop/data_BusStop" + str(i) + ".pkl"
  df_temp = pd.read_pickle(path)
  df_BusStop = pd.concat([df_BusStop, df_temp])

df_BusStop.to_pickle("/content/drive/MyDrive/Colab Notebooks/Capstone/BusStop/df_BusStop_all.pkl")
